In [1]:
import requests
import unicodedata

import pandas as pd
from bs4 import BeautifulSoup

scraped_data = pd.DataFrame(columns=['title', 'salary', 'desc', 'company_name', 'company_address'])

## Сбор данных с сайта [rabota.ru](https://irkutsk.rabota.ru):

In [2]:
for page in range(1, 94+1):
    url = f'https://irkutsk.rabota.ru/vacancy/?location.ll=52.289588,104.280606&location.kind=region&location.radius=any&location.regionId=121&location.name=%D0%98%D1%80%D0%BA%D1%83%D1%82%D1%81%D0%BA&sort=distance&page={page}'
    req = requests.get(url)
    if not req:
        raise Exception(f'Status code != 200: {req}')
    
    html = req.text
    bs = BeautifulSoup(html, 'html')
    for node in bs.find_all('div', {'class': 'vacancy-preview-card__top'}):
        content = node.find('div', {'class': 'vacancy-preview-card__content-wrapper'})
        content_title = content.find('a', {'class' : 'vacancy-preview-card__title_border'}).text.strip()
        content_salary = content.find('div', {'class': 'vacancy-preview-card__salary'}).text.strip()
        normalized_conent_salary = unicodedata.normalize("NFKD", content_salary)
        content_desc = content.find('div', {'class': 'vacancy-preview-card__short-description'}).text.strip()

        info = node.find('div', {'class': 'vacancy-preview-card__company-and-location vacancy-preview-card__company-and-location_height'})
        try:
            info_company_name = info.find('span', {'class': 'vacancy-preview-card__company-name'}).text.strip()
        except AttributeError:
            info_company_name = ''
        info_company_address = info.find('span', {'class': 'vacancy-preview-location__address-text'}).text.strip()

        scraped_data.loc[len(scraped_data)] = [content_title, normalized_conent_salary, content_desc, info_company_name, info_company_address]

scraped_data

,title,salary,desc,company_name,company_address
0,Техник-геофизик,от 110 500 руб.,Установка (размотка) геофизической аппаратуры ...,ООО ЗПГ,Иркутская область
1,Мастер по ремонту крупногабаритной бытовой тех...,от 80 000 руб.,Требуется мастер по ремонту крупногабаритной б...,"Алика-Сервис, ООО","Иркутск,"
2,Курьер пеший или вело,до 75 000 руб.,В команду партнера сервиса Яндекс.Еда срочно т...,Яндекс GO / Яндекс Еда - официальный партнер,"Иркутск, ул Желябова, д 1"
3,Водитель-курьер с личным автомобилем,от 85 000 руб.,Работа Курьером в Яндекс.Доставка (Оплата кажд...,ИП Розенфельд Анастасия Викторовна,"Иркутск, ул Ленина, д 8"
4,Менеджер по работе с клиентами,40 000 — 100 000 руб.,Требуемый опыт работы: 1–3 года; Полная занято...,Цифра Иркутск,"Иркутск, ул Горького, д 15"
...,...,...,...,...,...
1875,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа
1876,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа
1877,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа
1878,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа


## Добавление категорий к вакансиям:

In [3]:
categories = []
for title in scraped_data.title:
    if 'бухгалтер' in title.lower():
        categories.append('бухгалтер')
    elif 'продавец' in title.lower():
        categories.append('продавец')
    elif 'охранник' in title.lower():
        categories.append('охранник')
    elif 'оператор' in title.lower():
        categories.append('оператор')
    elif 'грузчик' in title.lower():
        categories.append('грузчик')
    elif 'руководитель' in title.lower():
        categories.append('руководитель')
    elif 'водитель' in title.lower():
        categories.append('водитель')
    elif 'менеджер' in title.lower():
        categories.append('менеджер')
    elif 'программист' in title.lower() or 'developer' in title.lower() or 'разработчик' in title.lower():
        categories.append('программист')
    elif 'монтажник' in title.lower():
        categories.append('монтажник')
    elif 'инжинер' in title.lower():
        categories.append('инжинер')
    elif 'курьер' in title.lower():
        categories.append('курьер')
    elif 'машинист' in title.lower():
        categories.append('машинист')
    elif 'администратор' in title.lower():
        categories.append('администратор')
    else:
        categories.append('другое')
    
scraped_data['categories'] = pd.Series(categories)
scraped_data

,title,salary,desc,company_name,company_address,categories
0,Техник-геофизик,от 110 500 руб.,Установка (размотка) геофизической аппаратуры ...,ООО ЗПГ,Иркутская область,другое
1,Мастер по ремонту крупногабаритной бытовой тех...,от 80 000 руб.,Требуется мастер по ремонту крупногабаритной б...,"Алика-Сервис, ООО","Иркутск,",другое
2,Курьер пеший или вело,до 75 000 руб.,В команду партнера сервиса Яндекс.Еда срочно т...,Яндекс GO / Яндекс Еда - официальный партнер,"Иркутск, ул Желябова, д 1",курьер
3,Водитель-курьер с личным автомобилем,от 85 000 руб.,Работа Курьером в Яндекс.Доставка (Оплата кажд...,ИП Розенфельд Анастасия Викторовна,"Иркутск, ул Ленина, д 8",водитель
4,Менеджер по работе с клиентами,40 000 — 100 000 руб.,Требуемый опыт работы: 1–3 года; Полная занято...,Цифра Иркутск,"Иркутск, ул Горького, д 15",менеджер
...,...,...,...,...,...,...
1875,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор
1876,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор
1877,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор
1878,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор


## Унифиция заголовков вакансий:

In [5]:
def replace_title(substr: str, replacestr: str) -> None:
    scraped_data.loc[
        scraped_data['title'].str.contains(substr, case=False),
        'title'
    ] = replacestr

to_replace = [
    ('Специалист по продвижению услуг', 'Специалист по продвижению услуг'),
    ('Оператор контакт-центра', 'Оператор контакт-центра'),
    ('Оператор контактного центра', 'Оператор контакт-центра'),
    ('Оператор call-центра', 'Оператор call-центра'),
    ('Специалист контакт-центра', 'Специалист контакт-центра'),
    ('Курьер', 'Курьер'),
    ('Менеджер интернет-магазина', 'Менеджер интернет-магазина'),
    ('Продавец-консультант', 'Продавец-консультант')
]

for i in to_replace:
    replace_title(i[0], i[1])

## Добавление столбцов минимальной и максимальной ЗП

In [6]:
min_salary = []
max_salary = []
for s in scraped_data['salary']:
    if s == 'по договоренности':
        min_salary.append(None)
        max_salary.append(None)
    elif '—' in s:
        salary = s.split('—')
        min_salary.append(salary[0].replace('руб.', '').replace(' ', ''))
        max_salary.append(salary[1].replace('руб.', '').replace(' ', ''))
    elif 'от' in s:
        salary = s.split('от')
        min_salary.append(salary[1].replace('руб.', '').replace(' ', ''))
        max_salary.append(None)
    elif 'до' in s:
        salary = s.split('до')
        min_salary.append(None)
        max_salary.append(salary[1].replace('руб.', '').replace(' ', ''))
    else:
        min_salary.append(None)
        max_salary.append(None)

scraped_data['min_salary'] = pd.Series(min_salary)
scraped_data['max_salary'] = pd.Series(max_salary)
scraped_data

,title,salary,desc,company_name,company_address,categories,min_salary,max_salary
0,Техник-геофизик,от 110 500 руб.,Установка (размотка) геофизической аппаратуры ...,ООО ЗПГ,Иркутская область,другое,110500,None
1,Мастер по ремонту крупногабаритной бытовой тех...,от 80 000 руб.,Требуется мастер по ремонту крупногабаритной б...,"Алика-Сервис, ООО","Иркутск,",другое,80000,None
2,Курьер,до 75 000 руб.,В команду партнера сервиса Яндекс.Еда срочно т...,Яндекс GO / Яндекс Еда - официальный партнер,"Иркутск, ул Желябова, д 1",курьер,None,75000
3,Курьер,от 85 000 руб.,Работа Курьером в Яндекс.Доставка (Оплата кажд...,ИП Розенфельд Анастасия Викторовна,"Иркутск, ул Ленина, д 8",водитель,85000,None
4,Менеджер по работе с клиентами,40 000 — 100 000 руб.,Требуемый опыт работы: 1–3 года; Полная занято...,Цифра Иркутск,"Иркутск, ул Горького, д 15",менеджер,40000,100000
...,...,...,...,...,...,...,...,...
1875,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор,30000,35000
1876,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор,30000,35000
1877,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор,30000,35000
1878,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор,30000,35000


## Преобразование адреса вакансии для дальнейшего определения координат:

In [ ]:
addresses = []
for address in scraped_data['company_address']:
    if 'Вахта в' in address:
        addresses.append(
            address.split('Вахта в')[1].strip()
        )
    elif 'Переезд в' in address:
        addresses.append(
            address.split('Переезд в')[1].strip()
        )
    else:
        addresses.append(address)
addresses

## Добавление координат для вакансий

In [8]:
import urllib.parse

lat = []
lon = []
for address in addresses:
    url = f'https://nominatim.openstreetmap.org/search/{urllib.parse.quote(address)}?format=json'
    response = requests.get(url).json()
    
    latitude = None
    longitude = None
    if response:
        latitude = response[0]["lat"]
        longitude = response[0]["lon"]
    lat.append(latitude)
    lon.append(longitude)

scraped_data['address_lat'] = pd.Series(lat)
scraped_data['address_lon'] = pd.Series(lon)
scraped_data

,title,salary,desc,company_name,company_address,categories,min_salary,max_salary,address_lat,address_lon
0,Техник-геофизик,от 110 500 руб.,Установка (размотка) геофизической аппаратуры ...,ООО ЗПГ,Иркутская область,другое,110500,None,56.6370122,104.719221
1,Мастер по ремонту крупногабаритной бытовой тех...,от 80 000 руб.,Требуется мастер по ремонту крупногабаритной б...,"Алика-Сервис, ООО","Иркутск,",другое,80000,None,52.289597,104.280586
2,Курьер,до 75 000 руб.,В команду партнера сервиса Яндекс.Еда срочно т...,Яндекс GO / Яндекс Еда - официальный партнер,"Иркутск, ул Желябова, д 1",курьер,None,75000,52.2865889,104.2816218
3,Курьер,от 85 000 руб.,Работа Курьером в Яндекс.Доставка (Оплата кажд...,ИП Розенфельд Анастасия Викторовна,"Иркутск, ул Ленина, д 8",водитель,85000,None,52.2779492,104.2845194
4,Менеджер по работе с клиентами,40 000 — 100 000 руб.,Требуемый опыт работы: 1–3 года; Полная занято...,Цифра Иркутск,"Иркутск, ул Горького, д 15",менеджер,40000,100000,52.2821099,104.2808625
...,...,...,...,...,...,...,...,...,...,...
1875,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор,30000,35000,None,None
1876,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор,30000,35000,None,None
1877,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор,30000,35000,None,None
1878,Оператор call-центра,30 000 — 35 000 руб.,Присоединяйтесь к федеральному аутсорсинговому...,Smarter,Удаленная работа,оператор,30000,35000,None,None


## Сохранение собранных данных:

In [10]:
csv_name = 'rabota_ru_07_01_23.csv'
scraped_data.to_csv(csv_name, index=False)
colab = False

if colab:
    from google.colab import files
    files.download(csv_name)